In [25]:
from requests import get
from bs4 import BeautifulSoup
import bs4
import os
import pandas as pd
import numpy as np

os.chdir('D:\\Learn\\py\\Kaggle\\CityofLA\\CityofLA')
bulletin_dir = "Job Bulletins"
additional_data_dir = 'Additional data'
competency_bank_dir = 'Competency Bank'

In [19]:
url = 'http://personnel.lacity.org/index.cfm?content=jobanalyses'
competency_urls=[]

response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

all_anchors = html_soup.find_all('a')
i=0
while (i<len(all_anchors)):
    if all_anchors[i].text == 'Competencies':
        competency_urls.append([all_anchors[i].get('title'), all_anchors[i].get('href')])
    #print(all_anchors[i].text)
    #print(all_anchors[i].get('title'),all_anchors[i].get('href'))
    i+=1;
#     if i == 3:
#         break

In [22]:
competency_urls

[['Accounting Clerk Competencies', 'eeo/AccountingClerk-CompetencyModel.pdf'],
 ['Accounting Records Supervisor Competencies',
  'eeo/AccountingRecordsSupervisor-CompetencyModel.pdf'],
 ['Administrative Analyst Competencies',
  'eeo/AdministrativeAnalyst-CompetencyModel.pdf'],
 ['Administrative Clerk Competencies',
  'eeo/AdministrativeClerk-CompetencyList.pdf'],
 ['Administrative Hearing Examiner Competencies',
  'eeo/AdministrativeHearingExaminer-CompetencyModel.pdf'],
 ['Airport Chief Information Security Officer Competencies',
  'eeo/AirportChiefInformationSecurityOfficer-CompetencyModel.pdf'],
 ['Airport Engineer Competencies', 'eeo/AirportEngineer-Competencies.pdf'],
 ['Airport Guide Competencies', 'eeo/AirportGuide-CompetencyList.pdf'],
 ['Airport Labor Relations Advocate Competencies',
  'eeo/AirportLaborRelationsAdvocate-CompetencyModel.pdf'],
 ['Airport Manager Competencies', 'eeo/AirportManager-CompetencyModel.pdf'],
 ['Airports Maintenance Supervisor Competencies',
  'eeo/A

In [20]:
len(competency_urls)

254

In [28]:
for url in competency_urls:
    file_url = 'http://personnel.lacity.org/'+url[1]
    file_name = url[0]+'.pdf'
  
    r = get(file_url, stream = True) 
  
    with open(os.path.join(competency_bank_dir,file_name),"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 
             if chunk:
                    pdf.write(chunk) 

In [33]:
import PyPDF2
import re

In [66]:
result = []
for filename in os.listdir(competency_bank_dir):
    pdfFileObj = open(os.path.join(competency_bank_dir,filename), 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #print(pdfReader.numPages)
    pageObj = pdfReader.getPage(0)
    txt = pageObj.extractText()
    numbers = []
    for lines in txt.split('\n'):
        if '.' in lines:
            number = lines.split('.')
            if number[0].strip().isdigit():
                numbers.append(number[0].strip())
    result.append([filename, ','.join(numbers)])
    #print(txt)

In [68]:
len(result)

254

In [69]:
df = pd.DataFrame(result, columns=['job_class', 'competency'])
s = df["competency"].str.split(',', expand=True).stack()
i = s.index.get_level_values(0)
df = df.loc[i].copy()
df["competency"] = s.values

In [73]:
df_competency_list = pd.read_csv('competencylist.csv')

In [74]:
df_competency_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
Id               60 non-null int64
SubCompetency    60 non-null object
Competency       60 non-null object
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


In [78]:
df['competency'] = df['competency'].apply(lambda x: 0 if x=='' else int(x))

In [80]:
df = df.merge(df_competency_list, how='inner', left_on='competency', right_on='Id')

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1566 entries, 0 to 1565
Data columns (total 5 columns):
job_class        1566 non-null object
competency       1566 non-null int64
Id               1566 non-null int64
SubCompetency    1566 non-null object
Competency       1566 non-null object
dtypes: int64(2), object(3)
memory usage: 73.4+ KB


In [84]:
df.drop(columns=['competency'], inplace=True)

In [85]:
df.to_csv('df_competency.csv')